<a href="https://colab.research.google.com/github/jstephens/tideprediction/blob/main/Tide_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries


In [1]:
!pip install keras --quiet
!pip install tensorflow --quiet
!pip install innvestigate --quiet

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import innvestigate
from bokeh.io import show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import DatetimeTickFormatter, DatetimeAxis
from bokeh.models import HoverTool
from bokeh.models import FixedTicker
from bokeh.embed import components
from bokeh.resources import CDN
from bokeh.embed import autoload_static
output_notebook()
from sklearn.model_selection import GridSearchCV
import imageio
import matplotlib.pyplot as plt
from datetime import datetime
import xgboost as xgb
import datetime
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.multioutput import RegressorChain, MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Data Load and Initial Transformation

In [4]:
X_train = np.load('drive/MyDrive/source/X_train_surge_new.npz')
X_test = np.load('drive/MyDrive/source/X_test_surge_new.npz')
Y_train = pd.read_csv('drive/MyDrive/source/Y_train_surge.csv',index_col = 'id_sequence')

df_X_test = pd.DataFrame.from_dict({item: X_test[item] for item in X_test.files}, orient='index').T.set_index('id_sequence')
df_X_train = pd.DataFrame.from_dict({item: X_train[item] for item in X_train.files}, orient='index').T.set_index('id_sequence')

In [5]:
def timestamp_to_datetime(timestamp):
    return pd.to_datetime('1970-01-01') + pd.to_timedelta(timestamp, unit='s')

df_X_train['t_slp'] = df_X_train['t_slp'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_train['t_surge1_input'] = df_X_train['t_surge1_input'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_train['t_surge2_input'] = df_X_train['t_surge2_input'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_train['t_surge1_output'] = df_X_train['t_surge1_output'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_train['t_surge2_output'] = df_X_train['t_surge2_output'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))

In [7]:
df_X_train

,t_slp,slp,t_surge1_input,surge1_input,t_surge2_input,surge2_input,t_surge1_output,t_surge2_output
id_sequence,,,,,,,,
1,"[1950-01-01 20:59:44, 1950-01-02 00:00:00, 195...","[[[102389.25, 102458.25, 102514.25, 102554.25,...","[1950-01-01 12:00:00, 1950-01-02 00:00:00, 195...","[-0.54593754, 0.13110517, 0.31880018, 0.194787...","[1950-01-01 14:00:32, 1950-01-02 01:59:28, 195...","[-0.41432458, -1.0094675, -1.2022604, -1.19387...","[1950-01-06 16:00:00, 1950-01-07 04:00:00, 195...","[1950-01-06 16:59:44, 1950-01-07 04:59:44, 195..."
2,"[1950-01-03 00:00:00, 1950-01-03 03:00:16, 195...","[[[101895.6, 101970.6, 102032.6, 102077.6, 102...","[1950-01-02 12:59:44, 1950-01-03 00:59:44, 195...","[0.31880018, 0.19478741, -0.22752637, -0.40181...","[1950-01-02 14:00:32, 1950-01-03 03:00:16, 195...","[-1.2022604, -1.193878, -1.1435843, -0.8166747...","[1950-01-07 16:00:00, 1950-01-08 04:00:00, 195...","[1950-01-07 17:59:28, 1950-01-08 06:00:32, 195..."
3,"[1950-01-04 00:00:00, 1950-01-04 03:00:16, 195...","[[[101661.24, 101700.24, 101774.24, 101849.24,...","[1950-01-03 14:00:32, 1950-01-04 01:59:28, 195...","[-0.22752637, -0.4018146, -0.87440383, -0.6397...","[1950-01-03 15:00:16, 1950-01-04 03:00:16, 195...","[-1.1435843, -0.81667477, -0.7579987, -0.12932...","[1950-01-08 16:59:44, 1950-01-09 04:59:44, 195...","[1950-01-08 17:59:28, 1950-01-09 07:00:16, 195..."
4,"[1950-01-05 00:00:00, 1950-01-05 03:00:16, 195...","[[[101366.41, 101393.41, 101476.41, 101577.41,...","[1950-01-04 14:00:32, 1950-01-05 01:59:28, 195...","[-0.87440383, -0.63978505, -1.0755056, -1.7123...","[1950-01-04 16:00:00, 1950-01-05 04:00:00, 195...","[-0.7579987, -0.12932655, -0.17962031, -0.3724...","[1950-01-09 16:59:44, 1950-01-10 06:00:32, 195...","[1950-01-09 19:00:16, 1950-01-10 07:00:16, 195..."
5,"[1950-01-06 00:00:00, 1950-01-06 03:00:16, 195...","[[[101245.49, 101220.49, 101209.49, 101225.49,...","[1950-01-05 15:00:16, 1950-01-06 03:00:16, 195...","[-1.0755056, -1.712328, 0.58693594, 1.0695802,...","[1950-01-05 16:00:00, 1950-01-06 04:59:44, 195...","[-0.17962031, -0.3724131, -0.05388589, 0.35684...","[1950-01-10 17:59:28, 1950-01-11 06:00:32, 195...","[1950-01-10 20:00:00, 1950-01-11 08:00:00, 195..."
...,...,...,...,...,...,...,...,...
5595,"[2010-10-19 08:59:44, 2010-10-19 11:58:56, 201...","[[[101470.0, 101486.0, 101501.0, 101517.0, 101...","[2010-10-18 23:00:16, 2010-10-19 10:59:12, 201...","[1.9376696, 1.6092033, 0.86177504, 1.2974956, ...","[2010-10-19 00:59:44, 2010-10-19 13:00:48, 201...","[0.021554768, 0.62508005, 0.39037576, 0.457434...","[2010-10-24 01:59:28, 2010-10-24 15:00:16, 201...","[2010-10-24 03:58:56, 2010-10-24 16:00:00, 201..."
5596,"[2010-10-20 08:59:44, 2010-10-20 12:01:04, 201...","[[[101663.87, 101650.87, 101641.87, 101635.87,...","[2010-10-20 00:00:00, 2010-10-20 12:01:04, 201...","[0.86177504, 1.2974956, 0.50984687, 0.87518185...","[2010-10-20 01:59:28, 2010-10-20 14:00:32, 201...","[0.39037576, 0.45743412, 0.28140593, 0.4322872...","[2010-10-25 01:59:28, 2010-10-25 15:00:16, 201...","[2010-10-25 04:01:04, 2010-10-25 16:00:00, 201..."
5597,"[2010-10-21 08:59:44, 2010-10-21 11:58:56, 201...","[[[101876.15, 101838.15, 101801.15, 101765.15,...","[2010-10-21 00:00:00, 2010-10-21 13:00:48, 201...","[0.50984687, 0.87518185, 1.8505255, 1.193593, ...","[2010-10-21 01:59:28, 2010-10-21 14:00:32, 201...","[0.28140593, 0.43228725, 0.15567149, 0.2478767...","[2010-10-26 02:59:12, 2010-10-26 16:00:00, 201...","[2010-10-26 05:00:48, 2010-10-26 16:59:44, 201..."


In [8]:
Y_train

,surge1_t0,surge1_t1,surge1_t2,surge1_t3,surge1_t4,surge1_t5,surge1_t6,surge1_t7,surge1_t8,surge1_t9,surge2_t0,surge2_t1,surge2_t2,surge2_t3,surge2_t4,surge2_t5,surge2_t6,surge2_t7,surge2_t8,surge2_t9
id_sequence,,,,,,,,,,,,,,,,,,,,
1,0.586936,1.069580,0.767928,-0.100162,0.070775,-0.244285,-0.354891,-0.928031,-0.773853,-0.375001,-0.053886,0.356847,0.348464,0.264641,0.901696,0.449052,0.113760,-0.422707,-0.456236,-0.825057
2,0.767928,-0.100162,0.070775,-0.244285,-0.354891,-0.928031,-0.773853,-0.375001,-0.361594,-0.210768,0.348464,0.264641,0.901696,0.449052,0.113760,-0.422707,-0.456236,-0.825057,-0.992703,-0.992703
3,0.070775,-0.244285,-0.354891,-0.928031,-0.773853,-0.375001,-0.361594,-0.210768,0.288635,-0.726929,0.901696,0.449052,0.113760,-0.422707,-0.456236,-0.825057,-0.992703,-0.992703,-0.322119,-0.883733
4,-0.354891,-0.928031,-0.773853,-0.375001,-0.361594,-0.210768,0.288635,-0.726929,-0.576103,1.160076,0.113760,-0.422707,-0.456236,-0.825057,-0.992703,-0.992703,-0.322119,-0.883733,-0.473001,-0.422707
5,-0.773853,-0.375001,-0.361594,-0.210768,0.288635,-0.726929,-0.576103,1.160076,0.442813,0.305393,-0.456236,-0.825057,-0.992703,-0.992703,-0.322119,-0.883733,-0.473001,-0.422707,-0.473001,-0.531677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5595,1.451674,1.220407,0.556771,1.709754,0.483033,0.533309,1.448322,1.820360,1.662831,1.528763,0.759197,0.884931,0.298171,-0.397560,-0.540059,-0.498148,-0.498148,-0.405942,0.164054,0.331700
5596,0.556771,1.709754,0.483033,0.533309,1.448322,1.820360,1.662831,1.528763,1.284089,0.070775,0.298171,-0.397560,-0.540059,-0.498148,-0.498148,-0.405942,0.164054,0.331700,1.052577,1.220223
5597,0.483033,0.533309,1.448322,1.820360,1.662831,1.528763,1.284089,0.070775,0.888589,1.545521,-0.540059,-0.498148,-0.498148,-0.405942,0.164054,0.331700,1.052577,1.220223,1.446545,2.670360


# Visualization: Timestamp Graph



In [ ]:
def timestamp_graph(df_bokeh):
  rows = [100, 101, 102]
  data_sources = []

  for i, row in enumerate(rows):
      df_row = pd.DataFrame(df_bokeh.iloc[row]).T

      columns = ['t_slp', 't_surge1_input', 't_surge2_input', 't_surge1_output', 't_surge2_output']
      dfs = {}

      for column in columns:
          df = pd.DataFrame(df_row[column])
          df = df.explode(column).reset_index(drop=True)
          dfs[column] = df

      x = [15, 8, 1][i]
      x2 = x + 2
      x3 = x + 3

      source_row = ColumnDataSource(data=dict(
          x=pd.to_datetime(dfs['t_slp']['t_slp']),
          y=[x] * len(dfs['t_slp']),
          size=[10] * len(dfs['t_slp']),
          color=[colorvar] * len(dfs['t_slp'])
      ))
      source2_row = ColumnDataSource(data=dict(
          x=pd.to_datetime(dfs['t_surge1_input']['t_surge1_input']),
          y=[x2] * len(dfs['t_surge1_input']),
          size=[10] * len(dfs['t_surge1_input']),
          color=[colorvar] * len(dfs['t_surge1_input'])
      ))
      source3_row = ColumnDataSource(data=dict(
          x=pd.to_datetime(dfs['t_surge2_input']['t_surge2_input']),
          y=[x3] * len(dfs['t_surge2_input']),
          size=[10] * len(dfs['t_surge2_input']),
          color=[colorvar] * len(dfs['t_surge2_input'])
      ))
      source2_pred_row = ColumnDataSource(data=dict(
          x=pd.to_datetime(dfs['t_surge1_output']['t_surge1_output']),
          y=[x2] * len(dfs['t_surge1_output']),
          size=[10] * len(dfs['t_surge1_output']),
          color=['lightgray'] * len(dfs['t_surge1_output'])
      ))
      source3_pred_row = ColumnDataSource(data=dict(
          x=pd.to_datetime(dfs['t_surge2_output']['t_surge2_output']),
          y=[x3] * len(dfs['t_surge2_output']),
          size=[10] * len(dfs['t_surge2_output']),
          color=['lightgray'] * len(dfs['t_surge2_output'])
      ))

      data_sources.extend([source_row, source2_row, source3_row, source2_pred_row, source3_pred_row])


  p = figure(plot_width=800, plot_height=400, x_axis_label='Timestamp', y_axis_label='', toolbar_location=None, x_axis_type='datetime')

  y_ticks = [1,2.5,3,4,8,9.5,10,11,15,16.5,17,18]
  y_labels = ['Encoded Image Recorded','Row 102                                                               ','Surge Time, City 1','Surge Time, City 2',
              'Encoded Image Recorded','Row 101                                                               ','Surge Time, City 1','Surge Time, City 2',
              'Encoded Image Recorded','Row 100                                                               ','Surge Time, City 1','Surge Time, City 2']
  p.yaxis.ticker = FixedTicker(ticks=y_ticks)
  p.yaxis.major_label_overrides = {tick: label for tick, label in zip(y_ticks, y_labels)}

  p.ygrid.grid_line_color = 'white'

  sources = [
      source_row100, source2_row100, source3_row100, source2_pred_row100, source3_pred_row100,
      source_row101, source2_row101, source3_row101, source2_pred_row101, source3_pred_row101,
      source_row102, source2_row102, source3_row102, source2_pred_row102, source3_pred_row102
  ]

  for source in sources:
      p.circle('x', 'y', source=source, size='size', alpha=0.5,color='color')

  p.x_range=Range1d(start=pd.Timestamp('1950-11-06 12:30:00'), end=pd.Timestamp('1950-11-18 12:30:00'))

  p.y_range=Range1d(start=0, end=19)

  p.xaxis.major_tick_line_color = None
  p.xaxis.minor_tick_line_color = None
  p.yaxis.major_tick_line_color = None
  p.yaxis.minor_tick_line_color = None

  hover = HoverTool(
      tooltips=[
          ("Time", "@x{%F %T}")
      ],
      formatters={"@x": "datetime"},
      mode='mouse'
  )
  p.add_tools(hover)
  output_notebook()
  show(p)

In [ ]:
df_bokeh = df_X_train.copy()
timestamp_graph(df_bokeh)

The plot above represents three typical rows for the X training dataset. Each row represents 5 days but will overlap with other rows temporally. 

In [ ]:
script, div = components(p)
print(div)
print(script)

# Visualization: Encoded Image GIF

In [ ]:
def gif(df_X_train):
  # Retrieve the 40 images from the row of interest
  images = df_X_train['slp'][5599]

  # Create an empty list to store the image file names
  image_files = []

  # Loop over each image and save it as a file
  for i in range(len(images)):
      # Create the file name with a 4-digit index
      filename = f'image_{i:04d}.png'
      
      # Save the image as a file
      plt.imsave(filename, images[i], cmap='gray')
      
      # Add the file name to the list
      image_files.append(filename)

  # Use imageio to create the gif
  with imageio.get_writer('encodedimage.gif', mode='I') as writer:
      # Loop over each image file and add it to the gif
      for filename in image_files:
          image = imageio.imread(filename)
          writer.append_data(image)

In [ ]:
gif(df_X_train)

# Timestamp Transformations



In [9]:
def get_nth_day(timestamps):
    first_timestamp = timestamps[0]
    year_start = datetime.datetime(first_timestamp.year, 1, 1)
    return int((first_timestamp - year_start).days + 1)

def get_last_date(timestamps):
    last_timestamp = timestamps[-1]
    year_start = datetime.datetime(last_timestamp.year, 1, 1)
    return int((last_timestamp - year_start).days + 1)

def get_first_year(timestamps):
    return timestamps[0].year

def extract_hour(timestamps):
    return np.array([pd.Timestamp(ts, unit='s').hour for ts in timestamps])

In [10]:
# apply the functions to each element in the column using pandas' apply() function
df_X_train['startingdate'] = df_X_train['t_slp'].apply(lambda x: get_nth_day(x))
df_X_train['lastdate'] = df_X_train['t_slp'].apply(lambda x: get_last_date(x))
df_X_train['year'] = df_X_train['t_slp'].apply(lambda x: get_first_year(x))
df_X_train['t_slp'] = df_X_train['t_slp'].apply(lambda x: extract_hour(x))
df_X_train['t_surge1_input'] = df_X_train['t_surge1_input'].apply(lambda x: extract_hour(x))
df_X_train['t_surge2_input'] = df_X_train['t_surge2_input'].apply(lambda x: extract_hour(x))
df_X_train['t_surge1_output'] = df_X_train['t_surge1_output'].apply(lambda x: extract_hour(x))
df_X_train['t_surge2_output'] = df_X_train['t_surge2_output'].apply(lambda x: extract_hour(x))

# Dynamic Time Warping 

In [ ]:
def dtw_distance(s1, s2):
    n, m = len(s1), len(s2)
    dtw = np.zeros((n+1, m+1))
    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = np.linalg.norm(s1[i-1] - s2[j-1])
            dtw[i][j] = cost + min(dtw[i-1][j], dtw[i][j-1], dtw[i-1][j-1])
    return dtw[n][m]

def find_closest_sequences(df, idx, k=None):
    # Extract the sequence to compare
    seq = df.iloc[idx]["slp"]
    
    # Compute DTW distance between the sequence and all prior sequences in the DataFrame
    distances = []
    for i in range(idx):
        dist = dtw_distance(seq, df.iloc[i]["slp"])
        distances.append(dist)
    distances = np.array(distances)
    
    # Get the top k indices of most similar sequences
    if k is None:
        k = len(distances)
    else:
        k = min(k, len(distances))
    top_k_indices = np.argsort(distances)[:k]
    
    # Return the closest sequences as a list of tuples (index, distance)
    closest_sequences = [(i, distances[i]) for i in top_k_indices]
    return closest_sequences

In [ ]:
df_X_train

,t_slp,slp,t_surge1_input,surge1_input,t_surge2_input,surge2_input,t_surge1_output,t_surge2_output,startingdate,lastdate,year
id_sequence,,,,,,,,,,,
1,"[20, 0, 3, 6, 8, 12, 15, 17, 20, 0, 3, 6, 8, 1...","[[[102389.25, 102458.25, 102514.25, 102554.25,...","[12, 0, 12, 0, 14, 1, 14, 1, 15, 3]","[-0.54593754, 0.13110517, 0.31880018, 0.194787...","[14, 1, 14, 3, 15, 3, 16, 4, 16, 4]","[-0.41432458, -1.0094675, -1.2022604, -1.19387...","[16, 4, 16, 4, 16, 4, 16, 6, 17, 6]","[16, 4, 17, 6, 17, 7, 19, 7, 20, 8]",1,6,1950
2,"[0, 3, 6, 8, 12, 15, 17, 20, 0, 3, 6, 8, 12, 1...","[[[101895.6, 101970.6, 102032.6, 102077.6, 102...","[12, 0, 14, 1, 14, 1, 15, 3, 16, 4]","[0.31880018, 0.19478741, -0.22752637, -0.40181...","[14, 3, 15, 3, 16, 4, 16, 4, 16, 4]","[-1.2022604, -1.193878, -1.1435843, -0.8166747...","[16, 4, 16, 4, 16, 6, 17, 6, 19, 8]","[17, 6, 17, 7, 19, 7, 20, 8, 20, 9]",3,7,1950
3,"[0, 3, 6, 8, 12, 15, 17, 20, 0, 3, 6, 8, 12, 1...","[[[101661.24, 101700.24, 101774.24, 101849.24,...","[14, 1, 14, 1, 15, 3, 16, 4, 16, 4]","[-0.22752637, -0.4018146, -0.87440383, -0.6397...","[15, 3, 16, 4, 16, 4, 16, 4, 17, 6]","[-1.1435843, -0.81667477, -0.7579987, -0.12932...","[16, 4, 16, 6, 17, 6, 19, 8, 20, 8]","[17, 7, 19, 7, 20, 8, 20, 9, 22, 11]",4,8,1950
4,"[0, 3, 6, 8, 12, 15, 17, 20, 0, 3, 6, 8, 12, 1...","[[[101366.41, 101393.41, 101476.41, 101577.41,...","[14, 1, 15, 3, 16, 4, 16, 4, 16, 4]","[-0.87440383, -0.63978505, -1.0755056, -1.7123...","[16, 4, 16, 4, 16, 4, 17, 6, 17, 7]","[-0.7579987, -0.12932655, -0.17962031, -0.3724...","[16, 6, 17, 6, 19, 8, 20, 8, 20, 9]","[19, 7, 20, 8, 20, 9, 22, 11, 0, 12]",5,9,1950
5,"[0, 3, 6, 8, 12, 15, 17, 20, 0, 3, 6, 8, 12, 1...","[[[101245.49, 101220.49, 101209.49, 101225.49,...","[15, 3, 16, 4, 16, 4, 16, 4, 16, 6]","[-1.0755056, -1.712328, 0.58693594, 1.0695802,...","[16, 4, 16, 4, 17, 6, 17, 7, 19, 7]","[-0.17962031, -0.3724131, -0.05388589, 0.35684...","[17, 6, 19, 8, 20, 8, 20, 9, 22, 11]","[20, 8, 20, 9, 22, 11, 0, 12, 0, 12]",6,10,1950
...,...,...,...,...,...,...,...,...,...,...,...
5595,"[8, 11, 15, 17, 21, 0, 2, 6, 8, 12, 15, 17, 21...","[[[101470.0, 101486.0, 101501.0, 101517.0, 101...","[23, 10, 0, 12, 0, 13, 0, 13, 0, 14]","[1.9376696, 1.6092033, 0.86177504, 1.2974956, ...","[0, 13, 1, 14, 1, 14, 2, 15, 2, 15]","[0.021554768, 0.62508005, 0.39037576, 0.457434...","[1, 15, 1, 15, 2, 16, 4, 16, 5, 16]","[3, 16, 4, 16, 5, 16, 5, 17, 6, 18]",292,297,2010
5596,"[8, 12, 15, 17, 21, 0, 2, 6, 8, 11, 15, 17, 21...","[[[101663.87, 101650.87, 101641.87, 101635.87,...","[0, 12, 0, 13, 0, 13, 0, 14, 1, 15]","[0.86177504, 1.2974956, 0.50984687, 0.87518185...","[1, 14, 1, 14, 2, 15, 2, 15, 3, 16]","[0.39037576, 0.45743412, 0.28140593, 0.4322872...","[1, 15, 2, 16, 4, 16, 5, 16, 5, 17]","[4, 16, 5, 16, 5, 17, 6, 18, 7, 20]",293,298,2010
5597,"[8, 11, 15, 17, 21, 0, 2, 6, 8, 12, 15, 17, 21...","[[[101876.15, 101838.15, 101801.15, 101765.15,...","[0, 13, 0, 13, 0, 14, 1, 15, 1, 15]","[0.50984687, 0.87518185, 1.8505255, 1.193593, ...","[1, 14, 2, 15, 2, 15, 3, 16, 4, 16]","[0.28140593, 0.43228725, 0.15567149, 0.2478767...","[2, 16, 4, 16, 5, 16, 5, 17, 6, 17]","[5, 16, 5, 17, 6, 18, 7, 20, 8, 21]",294,299,2010


In [ ]:
matches_list = []

for y in range(5000,5599):
  matches = find_closest_sequences(df_X_train,y,k=5599)
  matches_list.append(matches)

In [ ]:
lowest_value = 999999999999999999999999
highest_value = 0

for item in matches_list:
    if isinstance(item, list):  # Check if it's a sublist
        for pair in item:
            if pair[1] < lowest_value:
                lowest_value = pair[1]
            if pair[1] > highest_value:
                highest_value = pair[1]
    else:  # It's a tuple
        if item[1] < lowest_value:
            lowest_value = item[1]
        if item[1] > highest_value:
            highest_value = item[1]

print("Lowest value:", lowest_value)
print("Highest value:", highest_value)

Lowest value: 83805.66455078125
Highest value: 5821887.0


## Visualization: Time Sequence Similarity with DTW

In [ ]:
import matplotlib.cm as cm
import matplotlib
import matplotlib.colors as colors
from bokeh.models import Slider, CustomJS
from bokeh.layouts import column

p = figure(plot_width=800, plot_height=400, x_axis_label='X', y_axis_label='Y',
           x_range=(1, 366), y_range=(1949, 2011), tools='hover')

# Define the initial matches to be plotted
initial_matches = matches_list[1]

colormap = matplotlib.colormaps['viridis']  # Choose the desired colormap
normalize = colors.Normalize(vmin=lowest_value, vmax=highest_value)

# Create a ColumnDataSource for the matches
for x in initial_matches:
  if x[0] != 0:
    start = df_X_train.loc[x[0], 'startingdate']
    end = df_X_train.loc[x[0], 'lastdate']
    year= df_X_train.loc[x[0], 'year']
    hex_code = colors.rgb2hex(colormap(normalize(x[1])))
    p.line([start,end],[year,year],color=hex_code,line_alpha=0.8)

show(p)

ValueError: ignored

In [ ]:

p = figure(plot_width=800, plot_height=400, x_axis_label='X', y_axis_label='Y',
           x_range=(1, 366), y_range=(1949, 2011), tools='hover')

colormap = matplotlib.cm.get_cmap('viridis')  # Choose the desired colormap
normalize = matplotlib.colors.Normalize(vmin=lowest_value, vmax=highest_value)

source = ColumnDataSource(data=dict(start=[], end=[], year=[], color=[]))

callback = CustomJS(args=dict(source=source, matches_list=matches_list), code="""
    const data = matches_list[cb_obj.value];
    const start = data.map(x => x[0] !== 0 ? df_X_train[x[0]]['startingdate'] : null);
    const end = data.map(x => x[0] !== 0 ? df_X_train[x[0]]['lastdate'] : null);
    const year = data.map(x => x[0] !== 0 ? df_X_train[x[0]]['year'] : null);
    const color = data.map(x => x[0] !== 0 ? rgbToHex(colormap(normalize(x[1]))) : null);
    
    source.data = {start: start, end: end, year: year, color: color};
    source.change.emit();
""")

slider = Slider(title="Match Index", start=0, end=len(matches_list)-1, step=1, value=0)
slider.js_on_change('value', callback)

p.segment(x0='start', y0='year', x1='end', y1='year', line_color='color', line_width=2, line_alpha=0.5, source=source)

# Add a reference line in gray
ref_start = df_X_train.loc[2600, 'startingdate']
ref_end = df_X_train.loc[2600, 'lastdate']
ref_year = df_X_train.loc[2600, 'year']
p.line([ref_start, ref_end], [ref_year, ref_year], line_color='gray', line_width=2, line_alpha=0.8)

layout = column(p, slider)
show(layout)

<ipython-input-36-652afee802ca>:4: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap = matplotlib.cm.get_cmap('viridis')  # Choose the desired colormap


ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "xs" value "xs", key "ys" value "ys" [renderer: GlyphRenderer(id='902258', ...)]


# Principal Component Analysis

In [ ]:
pca = PCA(n_components=10)

# Define a function to apply PCA to a single image
def apply_pca(image):
    flat_image = image.reshape(-1, image.shape[-1])
    pca.fit(flat_image)
    return pca.transform(flat_image)

# Apply the PCA model to each image in the dataframe
df_X_train['slp'] = df_X_train['slp'].apply(lambda images: np.stack([apply_pca(image) for image in images]))

# Training Dataset Prep

In [11]:
def flatten_array(arr):
    return np.concatenate(arr, axis=None)

df_X_train['slp'] = df_X_train['slp'].apply(flatten_array)

In [12]:
df_X_train_tail = df_X_train.tail(1000)
Y_train_tail = Y_train.tail(1000)

X = df_X_train_tail.to_numpy()
y = Y_train_tail.to_numpy()

In [14]:
def flatten_array(row):
    new_row = []
    for col in row:
        if isinstance(col, np.ndarray):
            for val in col:
                new_row.append(val)
        else:
            new_row.append(col)
    return pd.Series(new_row)

df_flat = df_X_train_tail.apply(flatten_array, axis=1)

In [15]:
# Convert DataFrames to numpy arrays
X = df_flat.to_numpy()
y = Y_train_tail.to_numpy()

y_reshaped = np.reshape(y, (1000, 20))

# X-Test Dataset Prep

All the data processing work done on the X-train dataset, repeated for X-test.

In [ ]:
df_X_test['t_slp'] = df_X_test['t_slp'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_test['t_surge1_input'] = df_X_test['t_surge1_input'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_test['t_surge2_input'] = df_X_test['t_surge2_input'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_test['t_surge1_output'] = df_X_test['t_surge1_output'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_test['t_surge2_output'] = df_X_test['t_surge2_output'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))

df_X_test['startingdate'] = df_X_test['t_slp'].apply(lambda x: get_nth_day(x))
df_X_test['lastdate'] = df_X_test['t_slp'].apply(lambda x: get_last_date(x))
df_X_test['year'] = df_X_test['t_slp'].apply(lambda x: get_first_year(x))
df_X_test['t_slp'] = df_X_test['t_slp'].apply(lambda x: extract_hour(x))
df_X_test['t_surge1_input'] = df_X_test['t_surge1_input'].apply(lambda x: extract_hour(x))
df_X_test['t_surge2_input'] = df_X_test['t_surge2_input'].apply(lambda x: extract_hour(x))
df_X_test['t_surge1_output'] = df_X_test['t_surge1_output'].apply(lambda x: extract_hour(x))
df_X_test['t_surge2_output'] = df_X_test['t_surge2_output'].apply(lambda x: extract_hour(x))
df_X_test['slp'] = df_X_test['slp'].apply(lambda images: np.stack([apply_pca(image) for image in images]))

# Model

In [ ]:
base_regressor = RandomForestRegressor()

# Create the MultiOutputRegressor
multioutput_regressor = MultiOutputRegressor(base_regressor)

# Fit the model
multioutput_regressor.fit(X, y)

In [ ]:
param_grid = {
    'estimator__max_depth': [None, 5, 10],
    'estimator__min_samples_split': [2, 5, 10],
}

# Create the MultiOutputRegressor
base_regressor = DecisionTreeRegressor()
multioutput_regressor = MultiOutputRegressor(base_regressor)

# Create the GridSearchCV object
grid_search = GridSearchCV(multioutput_regressor, param_grid, cv=5)

# Fit the GridSearchCV object
grid_search.fit(X, y)

# Access the best estimator and its associated properties
best_estimator = grid_search.best_estimator_
best_params = grid_search.best_params_
best_score = grid_search.best_score_

In [ ]:
model_parameters = base_regressor.get_params()

model_parameters

{'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [ ]:
multioutput_regressor.estimators_[0].feature_importances_

len(multioutput_regressor.estimators_[1].feature_importances_)
print(len(df_flat.columns))

16503
